calculamos los distintos features:

In [1]:
import numpy as np
import seaborn as sns
import scipy.io as sio
import pandas as pd

from math import ceil, log
from random import randint
from scipy.stats import ranksums, bartlett, describe
from scipy.signal import welch
from scipy.fftpack import rfft
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d


plt.rcParams['figure.figsize'] = (10, 6)

In [2]:
filenamesP = ['../tp2/dataset/P0'+str(i)+'.mat' for i in range(1,10)]+['../tp2/dataset/P10.mat']
filenamesS = ['../tp2/dataset/S0'+str(i)+'.mat' for i in range(1,10)]+['../tp2/dataset/S10.mat']
bandas = ['delta_mean', 'epsilon_mean', 'alfa_mean', 'beta_mean', 'gamma_mean',
          'delta_std',  'epsilon_std',  'alfa_std',  'beta_std',  'gamma_std']

In [3]:
def FrecuenciasPorTipo(archivos, grupo):
    freqPotXPersona = []
    for filename in archivos:
        array = sio.loadmat(filename)['data']
        ffs = []
        for i in range(256):
            muestras= array[:,i][0]
            ff= welch(muestras, fs=250, nperseg=201)
            ffs.append(ff[1])
        frecuencia = ff[0]
        arrayffs = np.array(ffs)
        freqPotXPersona.append((frecuencia,arrayffs.mean(axis=0)))
#     for index, freqPot in enumerate(freqPotXPersona):
#         plt.plot(freqPot[0],freqPot[1], label=grupo+str(index+1))
#     plt.legend(loc='best')
#     plt.xlim(0,50)
#     plt.show()
    
    return freqPotXPersona

def PromedioBandas(freqPotXPersona):
    proms = []
    stds = []
    for j in range(len(freqPotXPersona)):
        proms.append([])
        stds.append([])
        freqPot = freqPotXPersona[j]
        buckets = [[],[],[],[],[]]
        for i in range(len(freqPot[1])):
            if freqPot[0][i] < 4:
                buckets[0].append(freqPot[1][i])
            elif freqPot[0][i] < 8:
                buckets[1].append(freqPot[1][i])
            elif freqPot[0][i] < 13:
                buckets[2].append(freqPot[1][i])
            elif freqPot[0][i] < 30:
                buckets[3].append(freqPot[1][i])
            else:
                buckets[4].append(freqPot[1][i])
        
        for i in range(len(buckets)):
            buckets[i] = np.array(buckets[i])
            proms[j].append(buckets[i].mean())
            stds[j].append(buckets[i].std())

    return proms, stds
    

In [4]:
frecuenciasP = FrecuenciasPorTipo(filenamesP,'P') #FrecuenciasPorTipo plotea
frecuenciasS = FrecuenciasPorTipo(filenamesS, 'S')

bandasP_mean, bandasP_std = PromedioBandas(frecuenciasP)
bandasS_mean, bandasS_std= PromedioBandas(frecuenciasS)

for i in range(len(bandasP_mean)):
    bandasP_mean[i] += bandasP_std[i]
    bandasS_mean[i] += bandasS_std[i]

In [5]:
# tablaP = pd.DataFrame(bandasP_mean, columns=bandas, index=['P'+str(i) for i in range(1,11)])
# tablaS = pd.DataFrame(bandasS_mean, columns=bandas, index=['S'+str(i) for i in range(1,11)])
# tablaP['Tipo'] = 'P'
# tablaS['Tipo'] = 'S'

# bandas_no_normalizadas = pd.concat([tablaP, tablaS])
# bandas_no_normalizadas.to_pickle("poder_espectral_no_norm.df")
bandas_no_normalizadas = pd.read_pickle("poder_espectral_no_norm.df")
bandas_no_normalizadas

,delta_mean,epsilon_mean,alfa_mean,beta_mean,gamma_mean,delta_std,epsilon_std,alfa_std,beta_std,gamma_std,Tipo
P1,1.013976e-11,2.114767e-12,8.597311e-13,1.357580e-12,2.591455e-13,6.045476e-12,7.788568e-13,1.333585e-13,6.366335e-13,6.654151e-13,P
P2,2.776192e-11,2.215931e-12,9.486518e-13,1.045988e-12,6.248615e-14,1.511155e-11,1.298956e-12,4.012490e-13,7.667534e-13,1.537834e-13,P
P3,4.401700e-11,3.088734e-12,4.638416e-13,1.285836e-13,9.613761e-15,3.942157e-11,1.782672e-12,2.064160e-13,7.283282e-14,2.408212e-14,P
P4,1.271719e-11,2.069442e-12,6.338850e-13,1.795954e-13,2.505703e-14,9.639446e-12,5.061183e-13,3.236940e-13,8.102794e-14,5.977254e-14,P
P5,1.064783e-11,1.700377e-12,9.192861e-13,9.914341e-13,1.323664e-13,7.996042e-12,4.632076e-13,2.727212e-13,2.757409e-13,3.181502e-13,P
P6,2.963112e-11,3.205336e-12,9.813531e-13,1.040890e-13,4.314605e-15,2.381545e-11,1.785273e-12,4.745956e-13,6.902684e-14,1.041625e-14,P
P7,3.774570e-11,3.035294e-12,8.245702e-13,3.797379e-13,1.783728e-14,3.252223e-11,1.687106e-12,1.848769e-13,1.867929e-13,3.952592e-14,P
P8,5.007863e-12,6.279532e-13,1.586481e-13,6.357998e-14,3.537354e-15,4.582997e-12,1.221504e-13,6.790768e-14,2.677355e-14,8.280548e-15,P
P9,2.225498e-11,4.619804e-12,1.409964e-12,9.723751e-13,8.611592e-14,1.286787e-11,1.077725e-12,2.246167e-13,2.590636e-13,2.191839e-13,P
P10,7.377688e-12,3.839709e-12,7.654888e-13,8.862251e-13,1.088521e-13,3.596349e-12,2.016596e-12,1.074745e-13,3.341737e-13,2.520218e-13,P


#### Bandas normalizadas

In [6]:
# bandas_normalizadas = bandas_no_normalizadas.copy(deep=True)

# for fila in bandas_normalizadas.index:
#     suma_mean = bandas_normalizadas.loc[fila][:5].sum()
#     suma_std =  bandas_normalizadas.loc[fila][5:-1].sum()
#     for col in bandas_normalizadas.columns:
#         if "mean" in col:
#             bandas_normalizadas.loc[fila][col] /= suma_mean
#         elif "std" in col:
#             bandas_normalizadas.loc[fila][col] /= suma_std

# bandas_normalizadas.to_pickle("poder_espectral_norm.df")
bandas_normalizadas = pd.read_pickle("poder_espectral_norm.df")
bandas_normalizadas

,delta_mean,epsilon_mean,alfa_mean,beta_mean,gamma_mean,delta_std,epsilon_std,alfa_std,beta_std,gamma_std,Tipo
P1,1.013976e-11,2.114767e-12,8.597311e-13,1.357580e-12,2.591455e-13,6.045476e-12,7.788568e-13,1.333585e-13,6.366335e-13,6.654151e-13,P
P2,2.776192e-11,2.215931e-12,9.486518e-13,1.045988e-12,6.248615e-14,1.511155e-11,1.298956e-12,4.012490e-13,7.667534e-13,1.537834e-13,P
P3,4.401700e-11,3.088734e-12,4.638416e-13,1.285836e-13,9.613761e-15,3.942157e-11,1.782672e-12,2.064160e-13,7.283282e-14,2.408212e-14,P
P4,1.271719e-11,2.069442e-12,6.338850e-13,1.795954e-13,2.505703e-14,9.639446e-12,5.061183e-13,3.236940e-13,8.102794e-14,5.977254e-14,P
P5,1.064783e-11,1.700377e-12,9.192861e-13,9.914341e-13,1.323664e-13,7.996042e-12,4.632076e-13,2.727212e-13,2.757409e-13,3.181502e-13,P
P6,2.963112e-11,3.205336e-12,9.813531e-13,1.040890e-13,4.314605e-15,2.381545e-11,1.785273e-12,4.745956e-13,6.902684e-14,1.041625e-14,P
P7,3.774570e-11,3.035294e-12,8.245702e-13,3.797379e-13,1.783728e-14,3.252223e-11,1.687106e-12,1.848769e-13,1.867929e-13,3.952592e-14,P
P8,5.007863e-12,6.279532e-13,1.586481e-13,6.357998e-14,3.537354e-15,4.582997e-12,1.221504e-13,6.790768e-14,2.677355e-14,8.280548e-15,P
P9,2.225498e-11,4.619804e-12,1.409964e-12,9.723751e-13,8.611592e-14,1.286787e-11,1.077725e-12,2.246167e-13,2.590636e-13,2.191839e-13,P
P10,7.377688e-12,3.839709e-12,7.654888e-13,8.862251e-13,1.088521e-13,3.596349e-12,2.016596e-12,1.074745e-13,3.341737e-13,2.520218e-13,P


####  Medida de la informacion intra-electrodo (entropia media)

In [7]:
tablaCantBines = pd.read_pickle('../tp2/bins.df')
tablaCantBines

,cant_bins,max,min
dataset/P01.mat,442.0,0.000020,-0.000020
dataset/P02.mat,398.0,0.000025,-0.000025
dataset/P03.mat,433.0,0.000028,-0.000029
dataset/P04.mat,418.0,0.000020,-0.000020
dataset/P05.mat,397.0,0.000027,-0.000027
dataset/P06.mat,449.0,0.000024,-0.000023
dataset/P07.mat,395.0,0.000023,-0.000024
dataset/P08.mat,472.0,0.000015,-0.000015
dataset/P09.mat,398.0,0.000026,-0.000026
dataset/P10.mat,421.0,0.000020,-0.000020


In [8]:
def discretizar(muestras, paciente_filename):
    N = tablaCantBines.at[paciente_filename,'cant_bins']
    maximo = tablaCantBines.at[paciente_filename,'max']
    minimo = tablaCantBines.at[paciente_filename,'min']
    bin_size = (maximo-minimo)/N
    discretizacion = np.array([0] * len(muestras))
    for i in range(len(muestras)):
        if muestras[i] < minimo:
            discretizacion[i] = 0
        elif muestras[i] > maximo:
            discretizacion[i] = N
        else:
            discretizacion[i] = int((muestras[i] - minimo) / bin_size)
    return discretizacion
def divideN(elem):
    return np.float32(elem)/201


def get_entropia(muestras, N):
    buckets =np.histogram(muestras, bins=int(N), range=(0.0, N))[0]
    assert np.sum(buckets) == 201
    buckets = np.apply_along_axis(func1d=divideN, axis=0, arr=buckets)
    # con eso nos quedan las frecuencias
    h = 0
    for value in buckets:
        if value > 0:
            h += value * log(value)
    return -h

In [14]:
def obtener_entropia_media(paciente_filename):
    paciente  = sio.loadmat('../tp2/' + paciente_filename)['data']
    cant_epochs = paciente.shape[0]
    cant_electrodos = paciente.shape[1]

    cantidad_mediciones = 10000
    entropia = []

    for a in range(cantidad_mediciones):
        epoch = randint(0,cant_epochs-1)
        electrodo = randint(0,cant_electrodos-1)
        disc = discretizar(paciente[epoch][electrodo], paciente_filename)
        entropia.append(get_entropia(disc, tablaCantBines.at[paciente_filename,'cant_bins']))
    
    entropia = np.array(entropia)
    return (entropia.mean(), entropia.std())

In [16]:
# for paciente_filename in tablaCantBines.index:
#     print paciente_filename
#     tablaCantBines.at[paciente_filename,"mean_entropy"], tablaCantBines.at[paciente_filename,"std_entropy"] = \
#     obtener_entropia_media(paciente_filename)
# tablaCantBines.to_pickle('bins_con_entropia.df')
tablaCantBines = pd.read_pickle('bins_con_entropia.df')
tablaCantBines

,cant_bins,max,min,mean_entropy,std_entropy
dataset/P01.mat,442.0,0.000020,-0.000020,4.559381,0.438055
dataset/P02.mat,398.0,0.000025,-0.000025,4.568171,0.363216
dataset/P03.mat,433.0,0.000028,-0.000029,4.513985,0.410765
dataset/P04.mat,418.0,0.000020,-0.000020,4.563222,0.418210
dataset/P05.mat,397.0,0.000027,-0.000027,4.589381,0.393151
dataset/P06.mat,449.0,0.000024,-0.000023,4.528059,0.456498
dataset/P07.mat,395.0,0.000023,-0.000024,4.588264,0.372754
dataset/P08.mat,472.0,0.000015,-0.000015,4.566092,0.422429
dataset/P09.mat,398.0,0.000026,-0.000026,4.597163,0.430269
dataset/P10.mat,421.0,0.000020,-0.000020,4.534647,0.409715


#### Medida de la informacion inter-electrodo